<a href="https://colab.research.google.com/github/Kevinlo937/objects_story/blob/main/cv_MultipleObjectDection3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 安裝 Ultralytics YOLOv8
!pip install ultralytics

# 安裝 Roboflow 套件（用於資料集下載）
!pip install roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 72.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 55.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 46.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

In [ ]:
# 檢查 CUDA 是否可用
!python -c "import torch; print(torch.cuda.is_available())"

# 檢查可用的 GPU 數量
!python -c "import torch; print(torch.cuda.device_count())"



True
1


In [ ]:
"""
修正後的資料集合併與訓練腳本

修正了原始腳本中未處理類別索引映射的問題，確保合併後的多個資料集
能夠正確訓練出多類別物件偵測模型。
"""

import os
import shutil
import yaml
from pathlib import Path
import roboflow
from ultralytics import YOLO

# --- 1. 環境設定與資料集下載 ---

# 設定 Roboflow API Key (請替換成您的金鑰)
API_KEY = "NKaOxmYYLFRPaf6w0uF6" # 請務必替換成您的有效金鑰
os.environ["ROBOFLOW_API_KEY"] = API_KEY

rf = roboflow.Roboflow(api_key=API_KEY)

# 定義要下載的資料集及其目標類別名稱
# 格式: (workspace, project, version, {original_class_name: target_class_name})
datasets_to_download = [
    ("package-detection", "package-at-front-door", 2, {"package": "package"}),
    ("findluggage", "find-luggage", 1, {"paper bag": "bag"}), # 根據需要取消註解
    ("project-ii", "person-7y27w", 2, {"person": "other_person"}),
    ("kevinlo937", "deliverman", 1, {"mailman": "delivery_worker"}),
    ("kevinlo937", "food_deliverman", 1, {"Foodpenda": "food_delivery"})
]

# 統一定義最終合併資料集的類別名稱和索引
# **這是關鍵步驟，確保所有來源的類別被映射到一致的索引**
UNIFIED_CLASSES = {
    "package": 0,          # 包裹 (來自 package-at-front-door)
    "bag": 1,              # 提袋 (如果使用 find-luggage)
    "other_person": 2,     # 其他人員 (來自 person-7y27w)
    "delivery_worker": 3,  # 郵差快遞員 (來自 deliverman)
    "food_delivery": 4     # 送餐員 (來自 food_deliverman)
}
# 反向映射，方便查找名稱
UNIFIED_CLASS_NAMES = {v: k for k, v in UNIFIED_CLASSES.items()}

print("開始下載資料集...")
downloaded_datasets_info = []
for workspace, project, version, class_mapping in datasets_to_download:
    try:
        print(f"正在下載: {workspace}/{project} v{version}")
        dataset = rf.workspace(workspace).project(project).version(version).download("yolov8")
        downloaded_datasets_info.append({
            "location": dataset.location,
            "original_yaml": Path(dataset.location) / "data.yaml",
            "class_mapping": class_mapping
        })
        print(f"下載完成: {dataset.location}")
    except Exception as e:
        print(f"下載失敗: {workspace}/{project} v{version}. 錯誤: {e}")

if not downloaded_datasets_info:
    print("沒有成功下載任何資料集，腳本終止。")
    exit()

print("所有資料集下載完成。")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
開始下載資料集...
正在下載: package-detection/package-at-front-door v2
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to package-at-front-door-2 in yolov8:: 100%|██████████| 2598/2598 [00:00<00:00, 10772.26it/s]

下載完成: /content/package-at-front-door-2
正在下載: findluggage/find-luggage v1
loading Roboflow workspace...


loading Roboflow project...



Extracting Dataset Version Zip to find-luggage-1 in yolov8:: 100%|██████████| 206/206 [00:00<00:00, 10676.22it/s]

...


Extracting Dataset Version Zip to deliverman-1 in yolov8:: 100%|██████████| 328/328 [00:00<00:00, 6198.19it/s]

下載完成: /content/deliverman-1
正在下載: kevinlo937/food_deliverman v1
loading Roboflow workspace...


loading Roboflow project...



Extracting Dataset Version Zip to food_deliverman-1 in yolov8:: 100%|██████████| 132/132 [00:00<00:00, 6921.55it/s]

下載完成: /content/food_deliverman-1
所有資料集下載完成。


In [ ]:
# --- 2. 資料集合併與標籤重新映射 ---

print("開始合併資料集並重新映射標籤...")
merged_dataset_dir = Path("merged_dataset")

# 創建合併資料集的目錄結構
for split in ["train", "valid", "test"]:
    os.makedirs(merged_dataset_dir / split / "images", exist_ok=True)
    os.makedirs(merged_dataset_dir / split / "labels", exist_ok=True)

# 遍歷每個下載的資料集進行合併和映射
for dataset_info in downloaded_datasets_info:
    dataset_path = Path(dataset_info["location"])
    original_yaml_path = dataset_info["original_yaml"]
    class_mapping = dataset_info["class_mapping"]

    print(f"處理資料集: {dataset_path.name}")

    # 讀取原始資料集的 data.yaml 以獲取原始類別名稱
    try:
        with open(original_yaml_path, 'r') as f:
            original_data_config = yaml.safe_load(f)
        original_class_names = original_data_config.get('names', [])
        if not original_class_names:
            print(f"警告: 無法從 {original_yaml_path} 讀取原始類別名稱，跳過此資料集。")
            continue
    except Exception as e:
        print(f"錯誤: 無法讀取或解析 {original_yaml_path}: {e}，跳過此資料集。")
        continue

    # 遍歷 train/valid/test 分割
    for split in ["train", "valid", "test"]:
        source_images_dir = dataset_path / split / "images"
        source_labels_dir = dataset_path / split / "labels"
        target_images_dir = merged_dataset_dir / split / "images"
        target_labels_dir = merged_dataset_dir / split / "labels"

        if not source_images_dir.exists() or not source_labels_dir.exists():
            print(f"警告: 資料集 {dataset_path.name} 的 {split} 分割不完整，跳過。")
            continue

        # 複製圖片檔案
        for img_file in source_images_dir.glob("*.*"):
            try:
                shutil.copy(img_file, target_images_dir / img_file.name)
            except Exception as e:
                print(f"複製圖片失敗: {img_file} -> {target_images_dir}. 錯誤: {e}")

        # 讀取、映射並寫入標籤檔案
        for label_file in source_labels_dir.glob("*.txt"):
            new_label_lines = []
            try:
                with open(label_file, 'r') as f_in:
                    lines = f_in.readlines()

                for line in lines:
                    parts = line.strip().split()
                    if not parts:
                        continue

                    original_class_index = int(parts[0])

                    # 檢查原始索引是否有效
                    if 0 <= original_class_index < len(original_class_names):
                        original_class_name = original_class_names[original_class_index]

                        # 檢查此類別是否需要被映射到目標類別
                        if original_class_name in class_mapping:
                            target_class_name = class_mapping[original_class_name]

                            # 檢查目標類別是否存在於統一定義中
                            if target_class_name in UNIFIED_CLASSES:
                                target_class_index = UNIFIED_CLASSES[target_class_name]
                                # 更新標籤行，使用新的統一索引
                                new_label_lines.append(f"{target_class_index} {' '.join(parts[1:])}\n")
                            else:
                                print(f"警告: 目標類別 '{target_class_name}' 未在 UNIFIED_CLASSES 中定義。來自檔案: {label_file}")
                        # else: # 如果原始類別不需要映射，可以選擇忽略或映射到 'other'
                        #     print(f"資訊: 原始類別 '{original_class_name}' 在 {label_file} 中未被映射，已忽略。")
                    else:
                        print(f"警告: 在 {label_file} 中發現無效的原始類別索引: {original_class_index}")

                # 將修改後的標籤寫入合併資料夾
                if new_label_lines:
                    target_label_file = target_labels_dir / label_file.name
                    with open(target_label_file, 'w') as f_out:
                        f_out.writelines(new_label_lines)

            except Exception as e:
                print(f"處理標籤檔案失敗: {label_file}. 錯誤: {e}")

print("資料集合併與標籤重新映射完成。")

開始合併資料集並重新映射標籤...
處理資料集: package-at-front-door-2
處理資料集: find-luggage-1
處理資料集: person-2
處理資料集: deliverman-1
處理資料集: food_deliverman-1
資料集合併與標籤重新映射完成。


In [ ]:
!grep -h -o '^[0-9]' merged_dataset/train/labels/*.txt | sort | uniq -c

   1905 0
     71 1
    168 2
    135 3
     56 4


In [ ]:
merged_dataset_dir='/content/merged_dataset'
print(merged_dataset_dir)

/content/merged_dataset


In [ ]:
# --- 3. 創建合併資料集的 data.yaml ---

print("創建合併資料集的 data.yaml...")
# Convert merged_dataset_dir to a Path object
merged_dataset_dir = Path(merged_dataset_dir)

merged_data_yaml_content = {
    'train': str(merged_dataset_dir / 'train' / 'images'),
    'val': str(merged_dataset_dir / 'valid' / 'images'),
    'test': str(merged_dataset_dir / 'test' / 'images'),
    'nc': len(UNIFIED_CLASSES),
    'names': {index: name for name, index in UNIFIED_CLASSES.items()} # 使用索引作為鍵
}

merged_data_yaml_path = merged_dataset_dir / "data.yaml"

try:
    with open(merged_data_yaml_path, 'w') as f:
        yaml.dump(merged_data_yaml_content, f, default_flow_style=False, sort_keys=False)
    print(f"合併後的 data.yaml 已創建: {merged_data_yaml_path}")
except Exception as e:
    print(f"創建合併後的 data.yaml 失敗: {e}")
    exit()


創建合併資料集的 data.yaml...
合併後的 data.yaml 已創建: /content/merged_dataset/data.yaml


In [ ]:
print(merged_data_yaml_path)

/content/merged_dataset/data.yaml


In [ ]:
# --- 4. 模型訓練 ---

print("開始模型訓練...")

# 載入預訓練模型 (例如 yolov8m.pt)
model = YOLO('yolov8m.pt')

# 開始訓練
try:
    results = model.train(
        data=str(merged_data_yaml_path), # 使用修正後的 data.yaml 路徑
        epochs=30,  # 根據需要調整訓練輪數
        imgsz=640,
        batch=16,   # 根據您的 GPU 記憶體調整
        patience=10,
        save=True,
        project="multi_class_detection", # 專案名稱
        name="merge_run",   # 訓練運行名稱
        # device=0, # 如果有 GPU，取消註解並指定 GPU 索引
        # workers=8 # 根據您的 CPU 核心數調整
    )
    print("模型訓練完成。結果保存在 'multi_class_detection/merge_run' 資料夾中。")

    # (可選) 進行驗證
    print("開始模型驗證...")
    validation_results = model.val()
    print("模型驗證完成。")

except Exception as e:
    print(f"模型訓練或驗證過程中發生錯誤: {e}")

開始模型訓練...
Ultralytics 8.3.124 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/content/merged_dataset/data.yaml, epochs=30, time=None, patience=10, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=multi_class_detection, name=merge_run4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_c

train: Scanning /content/merged_dataset/train/labels.cache... 1644 images, 3 backgrounds, 0 corrupt: 100%|██████████| 1647/1647 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 344.8±139.5 MB/s, size: 35.4 KB)


val: Scanning /content/merged_dataset/valid/labels.cache... 131 images, 1 backgrounds, 0 corrupt: 100%|██████████| 132/132 [00:00<?, ?it/s]


...


30 epochs completed in 0.497 hours.
Optimizer stripped from multi_class_detection/merge_run4/weights/last.pt, 52.0MB
Optimizer stripped from multi_class_detection/merge_run4/weights/best.pt, 52.0MB

Validating multi_class_detection/merge_run4/weights/best.pt...
Ultralytics 8.3.124 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 92 layers, 25,842,655 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:04<00:00,  1.18it/s]


                   all        132        137      0.898      0.888      0.946       0.82
               package         47         51      0.907      0.863      0.938      0.761
                   bag         19         21      0.992      0.905      0.949      0.905
          other_person         47         47          1      0.899      0.995      0.922
       delivery_worker         14         14          1      0.773      0.937      0.712
         food_delivery          4          4      0.594          1      0.912      0.801
Speed: 0.5ms preprocess, 11.0ms inference, 0.0ms loss, 7.2ms postprocess per image
Results saved to multi_class_detection/merge_run4
模型訓練完成。結果保存在 'multi_class_detection/merge_run' 資料夾中。
開始模型驗證...
Ultralytics 8.3.124 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 92 layers, 25,842,655 parameters, 0 gradients, 78.7 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1401.5±469.4 MB/s, size: 39.6 KB)


val: Scanning /content/merged_dataset/valid/labels.cache... 131 images, 1 backgrounds, 0 corrupt: 100%|██████████| 132/132 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:04<00:00,  1.99it/s]


                   all        132        137      0.899      0.888      0.946      0.822
               package         47         51      0.907      0.863      0.938      0.766
                   bag         19         21      0.992      0.905      0.949      0.905
          other_person         47         47          1      0.899      0.995      0.923
       delivery_worker         14         14          1      0.771      0.937      0.713
         food_delivery          4          4      0.598          1      0.912      0.801
Speed: 2.6ms preprocess, 21.4ms inference, 0.0ms loss, 2.9ms postprocess per image
Results saved to multi_class_detection/merge_run42
模型驗證完成。


In [ ]:
import yaml

# 讀取 data.yaml 檔案以獲取類別名稱
with open(f"{merged_dataset_dir}/data.yaml", 'r') as f:
    data = yaml.safe_load(f)
class_names = data['names']  # 獲取類別名稱字典

# Get precision and recall for each class
precision = validation_results.box.p
recall = validation_results.box.r

# Print precision and recall for each class with names
for i, (p, r) in enumerate(zip(precision, recall)):
    class_name = class_names[i]  # 獲取類別名稱
    print(f"Class {class_name:<20}: \tPrecision = {p:.4f}, \tRecall = {r:.4f}")

Class package             : 	Precision = 0.9069, 	Recall = 0.8627
Class bag                 : 	Precision = 0.9921, 	Recall = 0.9048
Class other_person        : 	Precision = 1.0000, 	Recall = 0.8985
Class delivery_worker     : 	Precision = 1.0000, 	Recall = 0.7715
Class food_delivery       : 	Precision = 0.5980, 	Recall = 1.0000


## 模型部署

### 1. 模型導出

In [ ]:
# 導出為 ONNX 格式（適用於多種推理框架）
model.export(format='onnx', dynamic=True, simplify=True)

# 導出為 TensorRT 格式（NVIDIA GPU 上的高性能推理）
model.export(format='engine', dynamic=True, simplify=True, device=0)

# 導出為 CoreML 格式（適用於 iOS 設備）
model.export(format='coreml', simplify=True)

# 導出為 TensorFlow SavedModel 格式
model.export(format='saved_model', simplify=True)

Ultralytics 8.3.124 🚀 Python-3.11.12 torch-2.6.0+cu124 CPU (Intel Xeon 2.00GHz)

PyTorch: starting from 'multi_class_detection/merge_run4/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 9, 8400) (49.6 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0', 'onnxslim>=0.1.46', 'onnxruntime-gpu'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 321.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.6/145.6 kB 326.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.8/280.8 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 201.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 306.3 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 22.0s, installed 3 packages: ['onnx>=1.12.0', 'onnxslim>=0.1.46', 'onnxruntime-gpu']
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect


ONNX: starting export w


CoreML: starting export with coremltools 8.3.0...


Running MIL backend_mlprogram pipeline: 100%|██████████| 12/12 [00:00<00:00, 88.08 passes/s]


CoreML: export success ✅ 27.4s, saved as 'multi_class_detection/merge_run4/weights/best.mlpackage' (49.5 MB)

Export complete (29.5s)
Results saved to /content/multi_class_detection/merge_run4/weights
Predict:         yolo predict task=detect model=multi_class_detection/merge_run4/weights/best.mlpackage imgsz=640  
Validate:        yolo val task=detect model=multi_class_detection/merge_run4/weights/best.mlpackage imgsz=640 data=/content/merged_dataset/data.yaml  
Visualize:       https://netron.app
Ultralytics 8.3.124 🚀 Python-3.11.12 torch-2.6.0+cu124 CPU (Intel Xeon 2.00GHz)

PyTorch: starting from 'multi_class_detection/merge_run4/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 9, 8400) (49.6 MB)
requirements: Ultralytics requirements ['sng4onnx>=1.0.1', 'onnx_graphsurgeon>=0.3.26', 'ai-edge-litert>=1.2.0', 'onnx2tf>=1.26.3'] not found, attempting AutoUpdate...
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━

100%|██████████| 1.11M/1.11M [00:00<00:00, 29.8MB/s]
Unzipping calibration_image_sample_data_20x128x128x3_float32.npy.zip to /content/calibration_image_sample_data_20x128x128x3_float32.npy...: 100%|██████████| 1/1 [00:00<00:00, 38.40file/s]


ONNX: starting export with onnx 1.17.0 opset 19...


ONNX: slimming with onnxslim 0.1.51...
ONNX: export success ✅ 8.1s, saved as 'multi_class_detection/merge_run4/weights/best.onnx' (98.8 MB)
TensorFlow SavedModel: starting TFLite export with onnx2tf 1.27.2...
Saved artifact at 'multi_class_detection/merge_run4/weights/best_saved_model'. The following endpoints are available:

* Endpoint 'serving_default'
  inputs_0 (POSITIONAL_ONLY): TensorSpec(shape=(1, 640, 640, 3), dtype=tf.float32, name='images')
Output Type:
  TensorSpec(shape=(1, 9, 8400), dtype=tf.float32, name=None)
Captures:
  136326922341456: TensorSpec(shape=(4, 2), dtype=tf.int32, name=None)
  136326922341264: TensorSpec(shape=(3, 3, 3, 48), dtype=tf.float32, name=None)
  136326922341840: TensorSpec(shape=(48,), dtype=tf.float32, name=None)
  136326922345296: TensorSpec(shape=(4, 2), dtype=tf.int32, name=None)
  136326922344912: TensorSpec(shape=(3, 3, 48, 96), dtype=tf.float32, name=None)
  136326922345872: TensorSpec(shape=(96,), dtype=tf.float32, name=None)
  13632692234

'multi_class_detection/merge_run4/weights/best_saved_model'

### 2. 部署到出入口即時包裹與人物意圖偵測系統

In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np
import time
from collections import defaultdict
from google.colab.patches import cv2_imshow
from IPython.display import clear_output  # << 重要：清除上個frame的顯示

In [ ]:
# 載入模型
model = YOLO('/content/multi_class_detection/merge_run4/weights/best.pt')

# 設定參數
conf_threshold = 0.5
video_source = 1
detect_interval = 15  # 每15張推論一次
snapshot_interval_sec = 5  # 每5秒存一張快照
snapshot_dir = "/content/snapshots"
os.makedirs(snapshot_dir, exist_ok=True)

if video_source == 0:
    def get_camera_stream(ip, username, password, channel=1):
        rtsp_url = f"rtsp://{username}:{password}@{ip}/Streaming/Channels/{channel}01"
        cap = cv2.VideoCapture(rtsp_url)
        return cap
    camera = get_camera_stream("106.107.183.134", "admin", "Qazwsx1122")
elif video_source == 1:
    video_path = "videoplayback.mp4"  # 替換成你的影片路徑
    camera = cv2.VideoCapture(video_path)
else:
    print("Invalid video source selected.")
    exit()

# 取得影片資訊
fps = camera.get(cv2.CAP_PROP_FPS) or 30  # 預設30fps
frame_width = int(camera.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(camera.get(cv2.CAP_PROP_FRAME_HEIGHT))
snapshot_interval_frames = int(fps * snapshot_interval_sec)

# 設定執行時間
end_time = time.time() + 300  # 最長5分鐘

frame_count = 0
inference_count = 0
latest_detected_classes = defaultdict(int)   # 最新一輪推論結果
cumulative_detected_classes = defaultdict(int)  # 累積所有推論結果
latest_names = []
frames = []  # 保存所有處理後的frame

# 主循環
while time.time() < end_time:
    ret, frame = camera.read()
    if not ret:
        print("無法讀取影像，可能是影片結束了。")
        break

    frame_count += 1

    # 每detect_interval張推論一次
    if frame_count % detect_interval == 0:
        results = model(frame, conf=conf_threshold)
        boxes = results[0].boxes.data.cpu().numpy() if results[0].boxes.data is not None else []
        latest_names = results[0].names
        inference_count += 1

        # 更新最新推論結果 + 累積推論統計
        latest_detected_classes = defaultdict(int)
        for det in boxes:
            _, _, _, _, score, class_id = det
            if score >= conf_threshold:
                class_name = latest_names[int(class_id)]
                latest_detected_classes[class_name] += 1
                cumulative_detected_classes[class_name] += 1

    # 畫推論資訊文字在frame上
    annotated_frame = frame.copy()

    # 左上角推論次數
    cv2.putText(annotated_frame, f"Inference Count: {inference_count}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # 右上角目前frame編號
    cv2.putText(annotated_frame, f"Frame: {frame_count}", (frame_width - 250, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 255), 2)

    # 左下角顯示累積偵測統計
    y_offset = 70
    for cls_name, count in cumulative_detected_classes.items():
        text = f"{cls_name}: {count}"
        cv2.putText(annotated_frame, text, (10, y_offset),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)
        y_offset += 40

    # 存進frames清單
    frames.append(annotated_frame)

    # 每隔snapshot_interval_frames儲存一張快照
    if frame_count % snapshot_interval_frames == 0:
        snapshot_path = os.path.join(snapshot_dir, f"snapshot_frame_{frame_count}.jpg")
        cv2.imwrite(snapshot_path, annotated_frame)
        print(f"儲存快照: {snapshot_path}")

camera.release()
print(f"總共處理了 {len(frames)} 張frame。")

# --- 合成影片 ---
output_video_path = '/content/inference_output.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

for frame in frames:
    out.write(frame)
out.release()

print(f"影片已儲存到 {output_video_path}")

# 🔥 🔥 🔥 在最後列印累積推論結果 🔥 🔥 🔥
print("\n===== 最後累積推論結果 =====")
for cls_name, count in cumulative_detected_classes.items():
    print(f"{cls_name}: {count} 次")
print("==============================")


0: 384x640 (no detections), 240.4ms
Speed: 8.6ms preprocess, 240.4ms inference, 15.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 80.0ms
Speed: 10.6ms preprocess, 80.0ms inference, 14.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 24.7ms
Speed: 2.5ms preprocess, 24.7ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 24.8ms
Speed: 2.8ms preprocess, 24.8ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 29.5ms
Speed: 3.5ms preprocess, 29.5ms inference, 4.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 39.7ms
Speed: 2.5ms preprocess, 39.7ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 24.7ms
Speed: 5.1ms preprocess, 24.7ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 24.7ms
Speed: 6.6ms preprocess, 24.

In [ ]:
# -*- coding: utf-8 -*-
from ultralytics import YOLO
import cv2
import os
import time
from collections import defaultdict

# 載入模型
# 請確保模型路徑正確
model_path = '/content/multi_class_detection/merge_run4/weights/best.pt'
if not os.path.exists(model_path):
    print(f"錯誤：找不到模型檔案 {model_path}。請確認路徑是否正確。")
    # 在 Colab 環境中，如果模型在 Google Drive，需要先掛載 Drive
    # from google.colab import drive
    # drive.mount('/content/drive')
    # model_path = '/content/drive/MyDrive/path/to/your/model/best.pt' # 修改為您在 Drive 中的實際路徑
    # if not os.path.exists(model_path):
    #     exit()
    exit() # 如果不在 Colab 或 Drive 中找不到，則退出

model = YOLO(model_path)

# 設定參數
conf_threshold = 0.5
video_source = 1  # 0: RTSP, 1: 本地影片檔案
detect_interval = 15  # 每15張推論一次
snapshot_interval_sec = 5  # 每5秒存一張快照
snapshot_dir = "/content/snapshots"
os.makedirs(snapshot_dir, exist_ok=True)

# --- 攝影機/影片來源設定 ---
if video_source == 0:
    # RTSP 來源設定 (請替換成您的實際資訊)
    rtsp_ip = "106.107.183.134"
    rtsp_user = "admin"
    rtsp_pass = "Qazwsx1122"
    rtsp_channel = 1
    rtsp_url = f"rtsp://{rtsp_user}:{rtsp_pass}@{rtsp_ip}/Streaming/Channels/{rtsp_channel}01"
    print(f"嘗試連接 RTSP: {rtsp_url}")
    camera = cv2.VideoCapture(rtsp_url)
elif video_source == 1:
    # 本地影片檔案設定 (請替換成您的影片路徑)
    video_path = "videoplayback.mp4"
    if not os.path.exists(video_path):
        print(f"錯誤：找不到影片檔案 {video_path}。請確認路徑是否正確。")
        # 在 Colab 環境中，如果影片在 Google Drive，需要先掛載 Drive
        # from google.colab import drive
        # drive.mount('/content/drive')
        # video_path = '/content/drive/MyDrive/path/to/your/video.mp4' # 修改為您在 Drive 中的實際路徑
        # if not os.path.exists(video_path):
        #     exit()
        exit() # 如果不在 Colab 或 Drive 中找不到，則退出
    print(f"讀取影片檔案: {video_path}")
    camera = cv2.VideoCapture(video_path)
else:
    print("錯誤：無效的影像來源選擇。")
    exit()

# 檢查攝影機/影片是否成功開啟
if not camera.isOpened():
    print("錯誤：無法開啟影像來源。請檢查路徑、網路連線或 RTSP 認證資訊。")
    exit()

# 取得影片資訊
fps = camera.get(cv2.CAP_PROP_FPS)
if fps is None or fps == 0:
    print("警告：無法取得影片 FPS，預設為 30。")
    fps = 30
frame_width = int(camera.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(camera.get(cv2.CAP_PROP_FRAME_HEIGHT))
snapshot_interval_frames = int(fps * snapshot_interval_sec)

print(f"影片資訊: {frame_width}x{frame_height} @ {fps:.2f} FPS")

# 設定執行時間 (例如：最長5分鐘)
max_duration_sec = 300
end_time = time.time() + max_duration_sec

# 初始化計數器和儲存變數
frame_count = 0
inference_count = 0
latest_detected_classes = defaultdict(int)   # 最新一輪推論結果
cumulative_detected_classes = defaultdict(int)  # 累積所有推論結果
latest_boxes_for_drawing = [] # 用於儲存最新偵測框，以便在非推論幀上繪製
latest_names = model.names # 從模型直接獲取類別名稱
frames = []  # 保存所有處理後的frame

print("開始處理影像...")
# 主循環
while time.time() < end_time:
    ret, frame = camera.read()
    if not ret:
        print("無法讀取影像，可能是影片結束或連線中斷。")
        break

    frame_count += 1
    annotated_frame = frame.copy() # 複製原始幀用於繪圖

    # 每detect_interval張推論一次
    if frame_count % detect_interval == 0:
        inference_count += 1
        print(f"執行第 {inference_count} 次推論 (Frame: {frame_count})")
        results = model(frame, conf=conf_threshold)
        # 獲取偵測框數據，如果沒有偵測到則為空列表
        boxes = results[0].boxes.data.cpu().numpy() if results[0].boxes.data is not None else []
        latest_boxes_for_drawing = boxes # 更新用於繪製的偵測框

        # 更新最新推論結果 + 累積推論統計
        latest_detected_classes = defaultdict(int)
        for det in boxes:
            # boxes 格式: [x1, y1, x2, y2, score, class_id]
            score = det[4]
            class_id = int(det[5])
            if score >= conf_threshold:
                class_name = latest_names[class_id]
                latest_detected_classes[class_name] += 1
                cumulative_detected_classes[class_name] += 1

    # --- 在畫布上繪製偵測框和標籤 (使用最新偵測結果) ---
    if latest_boxes_for_drawing is not None:
        for det in latest_boxes_for_drawing:
            x1, y1, x2, y2, score, class_id = det
            if score >= conf_threshold:
                class_id = int(class_id)
                class_name = latest_names[class_id]

                # 繪製偵測框 (綠色)
                color = (0, 255, 0)
                cv2.rectangle(annotated_frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)

                # 準備標籤文字
                label = f"{class_name}: {score:.2f}"

                # 計算文字大小和位置
                (label_width, label_height), baseline = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)
                # 確保標籤在圖片範圍內
                label_y = max(label_height + 10, int(y1) - 10)
                label_x = int(x1)

                # 繪製標籤背景
                cv2.rectangle(annotated_frame,
                              (label_x, label_y - label_height - baseline),
                              (label_x + label_width, label_y + baseline),
                              color, cv2.FILLED)
                # 繪製標籤文字 (黑色)
                cv2.putText(annotated_frame, label, (label_x, label_y),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
    # --- 繪製結束 ---

    # --- 在畫布上繪製其他資訊文字 ---
    # 左上角推論次數 (紅色)
    cv2.putText(annotated_frame, f"Inference Count: {inference_count}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # 右上角目前frame編號 (洋紅色)
    cv2.putText(annotated_frame, f"Frame: {frame_count}", (frame_width - 250, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 255), 2)

    # 左下角顯示累積偵測統計 (藍色)
    y_offset = frame_height - 30 # 從底部往上畫
    sorted_cumulative = sorted(cumulative_detected_classes.items(), key=lambda item: item[1], reverse=True)
    for cls_name, count in sorted_cumulative:
        text = f"{cls_name}: {count}"
        cv2.putText(annotated_frame, text, (10, y_offset),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)
        y_offset -= 30 # 往上移動繪製位置
        if y_offset < 60: # 避免覆蓋頂部文字
            break
    # --- 繪製結束 ---

    # 將處理後的幀加入列表
    frames.append(annotated_frame)

    # 每隔 snapshot_interval_frames 儲存一張快照
    if frame_count % snapshot_interval_frames == 0:
        snapshot_path = os.path.join(snapshot_dir, f"snapshot_frame_{frame_count}.jpg")
        try:
            cv2.imwrite(snapshot_path, annotated_frame)
            print(f"儲存快照: {snapshot_path}")
        except Exception as e:
            print(f"錯誤：無法儲存快照 {snapshot_path}: {e}")

    # (可選) 在 Colab 中顯示即時影像 (可能影響性能)
    # from google.colab.patches import cv2_imshow
    # cv2_imshow(cv2.resize(annotated_frame, (frame_width // 2, frame_height // 2))) # 縮小顯示
    # if cv2.waitKey(1) & 0xFF == ord('q'): # 在 Colab 中 waitKey 可能無法正常工作
    #    break

camera.release()
print(f"影像來源已關閉。總共處理了 {len(frames)} 張 frame。")

# --- 合成影片 ---
if frames:
    output_video_path = '/content/inference_output_with_boxes.mp4'
    print(f"開始合成影片到 {output_video_path}...")
    try:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v') # 或者使用 'XVID'
        out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

        for i, frame_to_write in enumerate(frames):
            out.write(frame_to_write)
            if (i + 1) % 100 == 0:
                print(f"已寫入 {i + 1}/{len(frames)} 幀到影片...")

        out.release()
        print(f"影片已成功儲存到 {output_video_path}")
    except Exception as e:
        print(f"錯誤：無法合成或儲存影片: {e}")
else:
    print("沒有處理任何幀，無法合成影片。")

# --- 在最後列印累積推論結果 ---
print("\n===== 最後累積推論結果 =====")
if cumulative_detected_classes:
    sorted_final_cumulative = sorted(cumulative_detected_classes.items(), key=lambda item: item[1], reverse=True)
    for cls_name, count in sorted_final_cumulative:
        print(f"{cls_name}: {count} 次")
else:
    print("沒有偵測到任何物件。")
print("==============================")

print("腳本執行完畢。")



讀取影片檔案: videoplayback.mp4
影片資訊: 1280x720 @ 29.97 FPS
開始處理影像...
執行第 1 次推論 (Frame: 15)

0: 384x640 (no detections), 111.4ms
Speed: 18.6ms preprocess, 111.4ms inference, 117.9ms postprocess per image at shape (1, 3, 384, 640)
執行第 2 次推論 (Frame: 30)

0: 384x640 (no detections), 25.1ms
Speed: 4.2ms preprocess, 25.1ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)
執行第 3 次推論 (Frame: 45)

0: 384x640 (no detections), 25.1ms
Speed: 3.8ms preprocess, 25.1ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)
執行第 4 次推論 (Frame: 60)

0: 384x640 (no detections), 25.2ms
Speed: 3.5ms preprocess, 25.2ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)
執行第 5 次推論 (Frame: 75)

0: 384x640 (no detections), 25.1ms
Speed: 3.9ms preprocess, 25.1ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)
執行第 6 次推論 (Frame: 90)

0: 384x640 (no detections), 25.1ms
Speed: 3.6ms preprocess, 25.1ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)
執行

In [ ]:
!zip -r /content/multi_class_detection_merge_run4.zip /content/multi_class_detection/merge_run4

  adding: content/multi_class_detection/merge_run4/ (stored 0%)
  adding: content/multi_class_detection/merge_run4/R_curve.png (deflated 10%)
  adding: content/multi_class_detection/merge_run4/val_batch0_pred.jpg (deflated 6%)
  adding: content/multi_class_detection/merge_run4/labels_correlogram.jpg (deflated 33%)
  adding: content/multi_class_detection/merge_run4/results.png (deflated 7%)
  adding: content/multi_class_detection/merge_run4/confusion_matrix_normalized.png (deflated 23%)
  adding: content/multi_class_detection/merge_run4/confusion_matrix.png (deflated 28%)
  adding: content/multi_class_detection/merge_run4/train_batch2062.jpg (deflated 10%)
  adding: content/multi_class_detection/merge_run4/val_batch2_pred.jpg (deflated 11%)
  adding: content/multi_class_detection/merge_run4/weights/ (stored 0%)
  adding: content/multi_class_detection/merge_run4/weights/best.pt (deflated 8%)
  adding: content/multi_class_detection/merge_run4/weights/best.onnx (deflated 17%)
  adding: con

In [ ]:
from google.colab import files
files.download('/content/multi_class_detection_merge_run4.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>